In [1]:
import os, sys
import numpy as np

import xarray as xr
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import colors

from scipy.fft import fft, ifft, fftfreq

plt.rc("figure", figsize=(16,8))
plt.rc("font", size=14)

from dask.diagnostics import ProgressBar
from dask.distributed import Client, LocalCluster
#
# Initialisation d'un cluster de 32 coeurs
cluster = LocalCluster(processes=False, n_workers=1, threads_per_worker=10)
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://137.129.156.19:8787/status,
Dashboard: http://137.129.156.19:8787/status,Workers: 1
Total threads: 10,Total memory: 94.31 GiB
Status: running,Using processes: False
Comm: inproc://137.129.156.19/47087/1,Workers: 1
Dashboard: http://137.129.156.19:8787/status,Total threads: 10
Started: Just now,Total memory: 94.31 GiB
Comm: inproc://137.129.156.19/47087/3,Total threads: 10
Dashboard: http://137.129.156.19:46813/status,Memory: 94.31 GiB
Nanny: None,


In [2]:
def isLeapYear (yearN):
    if ((yearN % 4 == 0) and (yearN % 100 != 0)) or (yearN % 400 == 0):
        reponse = True
    else:
        reponse = False
    print(reponse, '\n')
    return reponse

def hour_mean(x):
     return x.groupby('time.hour').mean('time')
    
def hour_sum(x):
     return x.groupby('time.hour').sum('time')
    
def hour_std(x):
     return x.groupby('time.hour').std('time')

In [3]:
indir = '/cnrm/tropics/commun/DATACOMMUN/ERA5/0.25/netcdf/sfc_6h/'
outdir = '/cnrm/tropics/commun/DATACOMMUN/WAVE/NO_SAVE/DATA/RAW_CLIM/'

ds = xr.open_mfdataset('/cnrm/tropics/commun/DATACOMMUN/ERA5/0.25/netcdf/sfc_3h/*.nc', chunks = {'time' : 2})

# group by dayofyear, then apply the function:
ds_clim = ds.groupby('time.dayofyear').apply(hour_mean).compute()
ds_clim.to_netcdf(outdir + 'clim_tcwv_brut_ERA5_3H_1979_2020.nc')

distributed.comm.inproc - WARNING - Closing dangling queue in <InProc  local=inproc://137.129.156.19/47087/1 remote=inproc://137.129.156.19/47087/9>


In [6]:
nbYear = 1
minYear = 1990
maxYear = 2020
year = np.arange(minYear,maxYear,nbYear)
print(year)

exp = 'era5_sfc_tcwv_0.25_'
indir = '/cnrm/tropics/commun/DATACOMMUN/ERA5/0.25/netcdf/sfc_3h/'
outdir = '/cnrm/tropics/commun/DATACOMMUN/WAVE/NO_SAVE/DATA/RAW_CLIM/'

datasets = []
for y in year :
    print(y)
    ds = xr.open_dataset(indir+exp+str(y)+'.nc', chunks = {'time' : 1}
                                 )
    datasets.append(ds)
        
ds = xr.concat(datasets, dim='time', coords='minimal', compat='override')
# group by dayofyear, then apply the function:
ds_clim = ds.groupby('time.dayofyear').apply(hour_mean).compute()
ds_clim.to_netcdf(outdir + 'clim_tcwv_brut_ERA5_3H_'+str(minYear)+'_'+str(maxYear)+'.nc')
print('end of script')

[1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003
 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017
 2018 2019]
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
